# Talktorial 12

# Binding Site Detection (DogSiteScorer)

Abishek Laxmanan Ravi Shankar

## Aim of the talktorial

To get a protein structure and **detect its ligand binding site** using **DoGSiteScorer** at the example of **EGFR** and validate the detected binding site with **KLIFS** and **ligand based detection**. 

Be more precise (can be done once you outlined the practical).

# Learning goals

### Theory
* Protein binding sites
* Binding site detection 
    * Introduction
    * DogSiteScorer    
* Validation
    * Ligand-based detection
    * KLIFS pocket definitions

### Practical

TBA

## References

* Combining global and local measures for structure-based druggability predictions.
([<i>Noordwijkerhout Cheminformatics</i> 2011, <b>52</b>, 360-372](https://www.ncbi.nlm.nih.gov/pubmed/22148551))

* Prediction, Analysis, and Comparison of Active Sites 
([ <i>Journal of Chemical Information and Modeling</i> 2018 <b>6,7</b>](https://onlinelibrary.wiley.com/doi/abs/10.1002/9783527806539.ch6g))

* DoGSiteScorer
([<i>Bioinformatics</i> 2012, <b>28</b>, 2074-2075](https://doi.org/10.1093/bioinformatics/bts310)) 

* Fpocket
([<i>BMC Bioinformatics</i> 2009, <b>10</b>, 168](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-10-168))

* Protein–ligand complex
([Protein complex wiki](https://en.wikipedia.org/wiki/Protein%E2%80%93ligand_complex))

* Voronoi tessellation
([Voronoi tessellation Wiki](https://en.wikipedia.org/wiki/Voronoi_diagram))

* KLIFS database
( [KLIFS Website](http://klifs.vu-compmedchem.nl/))  

* Fragments based drug Discovery
([Google books](https://books.google.de/books?id=tjfvCQAAQBAJ&pg=PA202&lpg=PA202&dq=Fragments+based+drug+Discovery+SURFNET&source=bl&ots=O6XVGMVPTV&sig=ACfU3U2K6btUeMpgN4FBe627jpIJL6-mtQ&hl=en&sa=X&ved=2ahUKEwjonezYmbLkAhXBRxUIHdzKClIQ6AEwAnoECAkQAQ))

* KLIFS: a structural kinase-ligand interaction database
([<i>KLIFS Pubmed</i> 2016, Nucleic Acids Research, <b>44</b>, 365-371](https://www.ncbi.nlm.nih.gov/pubmed/26496949)) 

* KLIFS: A Knowledge-Based Structural Database To Navigate Kinase−Ligand Interaction Space
([<i>KLIFS Journal of Medicinal Chemistry</i> 2013, <b>57</b>, 249-257](https://pubs.acs.org/doi/10.1021/jm400378w))

* SURFNET
([<i>Journal of Molecular Graphics</i>1995, <b>13</b>323-330](https://www.ncbi.nlm.nih.gov/pubmed/8603061))

* LIGSITE
([<i>Journal of Molecular Graphics and Modelling</i> 1997 <b>6</b> 359-363](https://www.sciencedirect.com/science/article/pii/S1093326398000023?via%3Dihub))

* Docking 
( [Docking wiki](https://en.wikipedia.org/wiki/Docking_(molecular)))

* DrugSite
([<i>Genome Informatics</i>2004 <b>15</b> 31–41 ](https://pdfs.semanticscholar.org/59ca/aac9a79c29660385af0ab83b82fba57cc476.pdf))

* Metapocket 2.0
([Summary of various binding site detection approaches](https://projects.biotec.tu-dresden.de/metapocket/algorithm.php))



## Theory

### Protein binding sites

The main objective of a pharmaceutical research is to find a new drug inorder to cure a specific disease. Structure-based approaches in drug design develop drugs with respect to a target protein which is associated with the disease under investigation. However, in order to develop such a drug it is crucial to know where the drug is supposed to bind in the protein and therefore inhibit or activate the protein. Not always such a small molecule binding site is known. Therefore it is important to have tools for binding site detection.

Binding site detection has a special mention in the process of drug development. Binding sites are defined as a **hollow 3 dimensional space** on the surface of a protein molecule, which serves as the region for **ligand**, **peptide**, or **protein** binding. There are various factors which influence whether such a region is suitable binding site.
* The Shape
* The Spatial arrangement of the residues
* The Possible interactions are
    * Van der Waals interactions
    * Hydrogen bond interactions
    * Hydrophobic regions
    * Electrostatic interactions
    * π-π interactions
   


### Binding site detection 

If ligand information is available (protein-ligand-complex), then the ligand-surrounding protein region is defined as pocket (e.g. using a 6 Å radius).
If the ligand is absent, then the detection tools can be used in the process of pocket detection. This comprises of 2 standard approaches. These methods can be grouped into, **geometry-based and energy-based** approaches as well as **grid-based and grid-free** approaches as outlined in the Figure 1.

<div style="width:500; font-size:100%; text-align:center;">
    <img src="images/Figure1_DetectionMethods.png" alt="alternate text"  width="500" style="padding:0.8em;"/>
    Figure 1: Binding site detection methods can be grouped into geometry-based and energy-based approaches as well as grid-based and grid-free approaches.
</div>

#### Geometry-based approaches

Geometry-based approaches analyze the shape of a molecular surface to locate cavities. It is based upon the 3D spatial arrangement of the atoms on the protein surface. They are broadly divided into two namely Grid-Based and Grid-free approaches.

* **Grid-based approaches**

    * Examples: POCKET, LIGSITE and DoGSiteScorer 
    * In LIGSITE, the cartesian coordinates runs along the X, Y and Z axes through a 3D region which consists of protein surface on both sides.
    * In addition to these axes a cubic diagonal passes along to find out the width of the cleft in between the protein surface. Therefore LIGSITE checks in seven directions the distance of each atom in the protein surface.
    * Therefore these coordinates enables LIGSITE to diffrentate the Protein Solvent Protein regions (PSP) on the protein surface
    * DoGSiteScorer is explained in detail below.
   
* **Grid-free approaches**

    * Example: SURFNET
    * In SURFNET spheres are placed on the protein surface instead of having grids. 
    * The probe spheres are placed midway between the pairs of atoms on the protein. Incase a probe overlaps any nearby atom, its radius is reduced until no overlap occurs. 
    * The resulting probes defines the pocket and the cavities. The cutoff radius of the spheres are of 1 to 4 angstrom .

#### Energy-based approaches

Interaction of a molecular fragment with protein is recorded. Favourable energitic responses are assigned to pockets. DrugSite and Docking are the well known energy-based methods. They are again divided into grid-based and grid-free approaches.
<div class="center" align="right" width="500"> </div>

* **Grid-based approaches**

    * Example: Drugsite
    * This method uses carbon probes placed on each grid point, and van der Wall's energies between the probes within a distance of 8 Å are calculated. Mean energy and Standard deviation are calculated for these grid points. 
    * Grid points with unfavourable energies are removed. Grid points fulfilling these cut-off are merged to pockets.
   
* **Grid-free approaches**

    * Example: Docking
    * Docking process is similar to "lock and key" model in which the **target protein resembles** the **lock** and **ligand** to the **key**
    * In this method, a scoring function is incorporated to evaluate the fragments that are placed at a position on the protein surface.
    * Scoring functions are physics based, which uses Molecular Mechanics force fields, which primarily estimates the amount of affinity between the binding ligand and the protein.
    * The scoring functions calculates the feasibilty of whether the ligand could be docked to the particular protein
    * These pockets are assigned based on the quantity of fragments that bind to a specific area.

### DogSiteScorer 

Certain geometric and physico-chemical properties are calculated in an automatic manner for the predicted pockets and subpockets. In **DogSiteScorer** , pockets and subpockets are predicted with DoGSite using a Difference of **Gaussian filter**. DogSiteScorer falls under the category of Geometry-based and Grid-based approaches. Pocket volume and surface are calculated by counting the **grid points** constituting the pocket volume or its surface and multiplying this number with the grid box volume or surface, respectively. A **breadth-first search** algorithm is used for pocket depth computation, starting from the solvent exposed pocket parts toward the most deeply buried regions. 

#### Pocket Detection

* The protein structure is covered around with a grid of coordinates.
* Each of the **grid points** are **labelled**.
* Difference of **Gaussian filter** is applied across the grid.
* This operation helps to find the position on a protein surface where the location of a **sphere-like object** is favorable.
* The last step in this algorithm is the merging of neighbouring subpockets to the pockets.

<div style="width:400; font-size:100%; text-align:center;">
    <img src="images/Figure2_BindingSitePocket.png" alt="alternate text"  width="400" style="padding:0.8em;"/>
    Figure 2: The binding site on the surface of the protein shown with grid view consisting of the surface molecules, volume inside the binding region, and the region exposed to the exterior of the protein 
</div>

#### Druggability 

* The Druggability is based on Machine learning technique - **Support Vector Machine (SVM)**.
* **Discriminative Analysis** - Best suited to seperate druggable from undruggable.
* The model has been trained and tested on both the redundant and non-redundant version of the druggable dataset
* This model showed a mean accuracy of 90%.
* The druggabilty score lies between 0 and 1. The higher the score the better the chance that the pocket is druggable.
    
<div style="width:1000; font-size:100%; text-align:center;">
    <img src="images/Figure3_DogSiteAlgo.png" alt="alternate text"  width="1000" style="padding:0.8em;"/>
    Figure 3: Flowsheet explaining the pipeline of DoGSiteScorer
</div>


### Validation methods

#### Ligand-based detection

Sometimes ligand information is available (protein-ligand-complex), in that case usually the ligand-surrounding protein region is defined as pocket (e.g. using a 6 Å radius).

If a bound ligand is known for a structure, it can be used for binding-site annotation.  


#### KLIFS pocket definition

KLIFS stands for **Kinase-Ligand Interaction Fingerprints and structures database**. It is a structural repository of over 2900 human and mouse kinase enzymes along with their catalytic activity. It also deals with the kinase domains and also the inhibitors which can interact with them. Kinases share a similar conserved regions, which serves as a challenge for small molecules that can selectively bind to a specific kinase. 

KLIFS enables us to do systematic comparison and analysis of chemical features of all available kinases in the process of ligand binding. The classification helps us of an all-encompassing binding site of 85 residues. It is possible to compare the **interaction patterns** of **kinase-inhibitors** to each other to, for example, identify crucial interactions determining kinase-inhibitor selectivity. Therefore this could be an ideal method used for the validation of the obtained binding site. 

## Practical

In [1]:
import requests
import time
from biopandas.pdb import PandasPdb
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Binding site detection using DoGSiteScorer

#### Submit job to DoGSiteScorer and get job location

In [3]:
def dogsite_scorer_submit_with_pdbid(pdb_code, chain_id, ligand=''):
    """
    Description
    -----------
    Submit PDB ID to DoGSiteScorer webserver using their API and get back URL for job location.
    This function is taken from talktorial 11b by @jaimergp.
    
    Parameters
    ----------
    pdb_id : str 
        4-letter valid PDB ID, e.g. '3w32'.
    chain : str
        Chain ID, e.g. 'A'.
    ligand : str
        Name of ligand bound to PDB structure with pdb_id, e.g. 'W32_A_1101'. 
        
    Returns
    -------
    str
        Job location URL for submitted query.
    """
    
    # Submit job to proteins.plus
    r = requests.post("https://proteins.plus/api/dogsite_rest",
        json={
            "dogsite": {
                "pdbCode": pdb_code,
                "analysisDetail": "1",
                "bindingSitePredictionGranularity": "1",
                "ligand": ligand,
                "chain": chain_id
            }
        },
        headers= {'Content-type': 'application/json', 'Accept': 'application/json'}
    )

    r.raise_for_status()
    
    return r.json()['location']

In [4]:
#Identifying the job location where the work is submitted to the web server
job_location = dogsite_scorer_submit_with_pdbid('3w32', 'A')
job_location

'https://proteins.plus/api/dogsite_rest/i2AH31qHweZuwnHxJnA824UM'

In [5]:
time.sleep(30)

#### Get DoGSiteScorer pocket metadata

In [6]:
def get_dogsitescorer_metadata(job_location):
    """
    Description
    -----------  
    This function retrieves results from a DoGSiteScorer submission.
    It returns the binding sites which are found over the protein surface, in the form of a table with the details about all detected pockets.

    Parameters
    ----------
    job_location : str
        Consists of the location where the "protein.plus (DoGSiteScorer)" is accessed
    
    Returns
    -------
    result_table_df : pandas.DataFrame
        Contains "Binding Site Data" returned from protein.plus web server in the form of Pandas DataFrame Table 
    """
    
    # Get job results
    result = requests.get(job_location)
    
    # Get result table
    result_table = requests.get(result.json()['result_table']).text
    
    # Split string into list of lists (=table)
    result_table_split = [i.split('\t') for i in result_table[:-1].split('\n')]

    # Remove spaces
    result_table_split = [[j.replace(' ', '') for j in i] for i in result_table_split]

    # Extract column names, index names, table body
    column_names = result_table_split[0]
    index_names = [i[0] for i in result_table_split[1:]]
    table = [i[1:] for i in result_table_split[1:]]

    # Convert to DataFrame
    result_table_df = pd.DataFrame(
        table,
        columns=column_names[1:],
        index=index_names
    )
    result_table_df.index.name = 'name'
    
    # Convert number strings to numeric values
    for name, data in result_table_df.iteritems():
        try:
            result_table_df[name] = pd.to_numeric(data)
        except ValueError:
            pass
    
    return result_table_df

#### Extract guessed binding site (according to DoGSiteScorer)

In [7]:
#Retrieving all the guessed pockets from DoGSiteScorer Web server 
metadata = get_dogsitescorer_metadata(job_location)
metadata

,lig_cov,poc_cov,lig_name,volume,enclosure,surface,depth,surf/vol,lid/hull,ellVol,ellc/a,ellb/a,siteAtms,accept,donor,hydrophobic_interactions,hydrophobicity,metal,Cs,Ns,Os,Ss,Xs,negAA,posAA,polarAA,apolarAA,ALA,ARG,ASN,ASP,CYS,GLN,GLU,GLY,HIS,ILE,LEU,LYS,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
P_0,0.0,0.0,"""""",1422.66,0.10,1673.75,19.26,1.176493,-,-,0.13,0.67,288,86,40,71,0.36,0,198,45,41,4,0,0.10,0.13,0.24,0.53,4,5,2,5,2,2,1,5,0,3,12,3,2,3,3,1,2,1,1,5,0.63,0.810023
P_0_0,0.0,0.0,"""""",599.23,0.06,540.06,17.51,0.901257,-,-,0.14,0.22,131,35,13,25,0.34,0,95,16,17,3,0,0.03,0.10,0.28,0.59,1,2,1,1,2,1,0,2,0,2,7,1,2,2,1,0,2,0,0,2,0.59,0.620201
P_0_1,0.0,0.0,"""""",201.73,0.08,381.07,11.36,1.889010,-,-,0.17,0.25,51,17,9,10,0.28,0,36,6,7,2,0,0.08,0.17,0.25,0.50,1,1,0,1,1,1,0,0,0,0,3,1,1,0,0,0,1,0,0,1,0.17,0.174816
P_0_2,0.0,0.0,"""""",185.60,0.17,282.00,9.35,1.519397,-,-,0.45,0.55,48,17,8,12,0.32,0,31,8,8,1,0,0.17,0.25,0.08,0.50,0,2,0,1,0,0,1,1,0,0,2,1,1,1,0,0,0,0,0,2,0.13,0.195695
P_0_3,0.0,0.0,"""""",175.30,0.15,297.42,9.29,1.696634,-,-,0.23,0.37,48,16,8,14,0.37,0,32,8,8,0,0,0.14,0.14,0.36,0.36,1,1,1,2,0,0,0,3,0,0,1,1,0,1,1,1,0,0,0,1,0.13,0.168845
P_0_4,0.0,0.0,"""""",170.37,0.08,390.10,11.99,2.289722,-,-,0.16,0.20,47,14,7,17,0.45,0,34,7,6,0,0,0.00,0.18,0.18,0.64,2,2,0,0,0,1,0,0,0,1,3,0,0,0,0,0,0,1,1,0,0.15,0.223742
P_0_5,0.0,0.0,"""""",90.43,0.24,177.50,6.24,1.962844,-,-,0.70,0.89,26,8,6,5,0.26,0,16,6,4,0,0,0.12,0.25,0.25,0.38,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,2,0.00,0.165232
P_1,0.0,0.0,"""""",708.99,0.13,1030.19,14.32,1.453039,-,-,0.14,0.59,140,44,13,34,0.37,0,98,17,25,0,0,0.14,0.11,0.36,0.39,3,1,1,0,0,0,4,4,0,1,4,2,0,1,1,2,2,0,1,1,0.46,0.755915
P_1_0,0.0,0.0,"""""",496.90,0.11,739.17,12.72,1.487563,-,-,0.14,0.18,103,34,8,22,0.34,0,74,12,17,0,0,0.18,0.09,0.32,0.41,2,1,1,0,0,0,4,1,0,1,4,1,0,0,1,2,2,0,1,1,0.49,0.465489


#### Arranging the pockets based on their drugScore

In [8]:
#Sorting the obtained binding site in the order of largest drugScore to the smallest
metadata.sort_values(by='drugScore', ascending=False)

,lig_cov,poc_cov,lig_name,volume,enclosure,surface,depth,surf/vol,lid/hull,ellVol,ellc/a,ellb/a,siteAtms,accept,donor,hydrophobic_interactions,hydrophobicity,metal,Cs,Ns,Os,Ss,Xs,negAA,posAA,polarAA,apolarAA,ALA,ARG,ASN,ASP,CYS,GLN,GLU,GLY,HIS,ILE,LEU,LYS,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
P_0,0.0,0.0,"""""",1422.66,0.10,1673.75,19.26,1.176493,-,-,0.13,0.67,288,86,40,71,0.36,0,198,45,41,4,0,0.10,0.13,0.24,0.53,4,5,2,5,2,2,1,5,0,3,12,3,2,3,3,1,2,1,1,5,0.63,0.810023
P_1,0.0,0.0,"""""",708.99,0.13,1030.19,14.32,1.453039,-,-,0.14,0.59,140,44,13,34,0.37,0,98,17,25,0,0,0.14,0.11,0.36,0.39,3,1,1,0,0,0,4,4,0,1,4,2,0,1,1,2,2,0,1,1,0.46,0.755915
P_0_0,0.0,0.0,"""""",599.23,0.06,540.06,17.51,0.901257,-,-,0.14,0.22,131,35,13,25,0.34,0,95,16,17,3,0,0.03,0.10,0.28,0.59,1,2,1,1,2,1,0,2,0,2,7,1,2,2,1,0,2,0,0,2,0.59,0.620201
P_3,0.0,0.0,"""""",244.16,0.04,514.94,14.34,2.109027,-,-,0.12,0.31,92,18,7,24,0.49,0,67,12,13,0,0,0.12,0.06,0.41,0.41,0,0,1,1,1,0,1,1,1,0,1,0,0,2,3,2,1,0,1,1,0.12,0.572013
P_4_1,0.0,0.0,"""""",60.93,0.20,193.96,5.80,3.183325,-,-,0.54,0.84,28,4,1,16,0.76,0,22,4,2,0,0,0.00,0.14,0.14,0.71,0,0,0,0,0,0,0,0,0,1,0,1,1,1,2,0,1,0,0,0,0.00,0.570708
P_2,0.0,0.0,"""""",286.21,0.18,462.29,11.83,1.615213,-,-,0.23,0.44,70,20,9,13,0.31,0,50,9,9,2,0,0.12,0.06,0.18,0.65,1,0,0,2,0,1,0,0,0,1,1,1,2,1,2,0,1,0,1,3,0.09,0.537137
P_1_0,0.0,0.0,"""""",496.90,0.11,739.17,12.72,1.487563,-,-,0.14,0.18,103,34,8,22,0.34,0,74,12,17,0,0,0.18,0.09,0.32,0.41,2,1,1,0,0,0,4,1,0,1,4,1,0,0,1,2,2,0,1,1,0.49,0.465489
P_4,0.0,0.0,"""""",169.28,0.21,373.47,11.49,2.206226,-,-,0.12,0.16,59,14,3,24,0.59,0,44,7,7,1,0,0.08,0.08,0.17,0.67,0,0,0,1,0,1,0,0,0,3,0,1,2,1,2,0,1,0,0,0,0.07,0.424397
P_5,0.0,0.0,"""""",166.59,0.16,347.46,11.99,2.085719,-,-,0.14,0.24,58,18,8,10,0.28,0,39,8,11,0,0,0.15,0.08,0.46,0.31,0,0,1,1,0,2,1,1,1,1,1,0,1,0,1,1,0,0,1,0,0.00,0.401384


In [9]:
def select_best_pocket(metadata, by='drugScore'):
    
    """
    Description
    -----------
    'drugScore' is used as a parameter to identify the best pocket among the obtained pockets
    
    Parameters
    ----------
    metadata : pd.DataFrame
    
    by : str
       
        
    Returns 
    -------
    best_pocket_name : str
        The best pocket based on the 'DrugScore' is returned here as the output
    
    """
    
    by_methods = ['drugScore', 'volume']
    
    if by == 'drugScore':
        # Sort by best druggability score
        best_pocket = metadata.sort_values(by='drugScore', ascending=False).iloc[0, :]
    elif by == 'volume':
        best_pocket = metadata.sort_values(by='volume', ascending=False).iloc[0, :]
    else:
        raise ValueError(f'Selection method not in list: {", ".join(by_methods)}')
                         
    # Get best pocket name
    best_pocket_name = best_pocket.name     
        
    return best_pocket_name

In [10]:
#To get the best "binding site name" from all obtained binding sites. 
best_pocket = select_best_pocket(metadata, by='drugScore')
best_pocket

'P_0'

#### Get best binding site file

In [11]:
def get_pocket_filenames(joblocation):
    
    """
    Description
    -----------
    We have identified the best pocket by the above step.
    Now we must obtain the best pocket file in ".pdb" format.
    
    Parameters
    ----------
    job_location : str
    
    Returns
    -------
    residues : str
    
    """
    
    # Get job results
    result = requests.get(job_location)
    
    # Get residues
    return result.json()['residues']

In [12]:
def get_best_pocket_filenames(pocket_files, best_pocket):
    
    """
    Description
    -----------
    To create a function to get the best binding site file from the list of all obtained pdb files.
    
    Parameters
    ----------
    pocket_files : list
    best_pocket : str

    Returns
    ------
    result[] = list
    """
    result = []
    
    for pocket_file in pocket_files:
        
        if f'{best_pocket}_res' in pocket_file:
            result.append(pocket_file)
            
    if len(result) > 1:
        raise TypeError(f'Multiple strings detected: {", ".join(result)}.')
    elif len(result) == 0:
        raise TypeError(f'No string detected.')
    else:
        pass
            
    return result[0]

In [13]:
# Get URL for all PDB files from the DoGSiteScorer Web server
pocket_files = get_pocket_filenames(job_location)
pocket_files

['https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_0_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_0_0_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_0_1_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_0_2_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_0_3_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_0_4_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_0_5_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_1_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_1_0_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_1_1_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_2_res.pdb',
 'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA8

In [14]:
# Get URL for PDB file for best binding site
best_pocket_file_location = get_best_pocket_filenames(pocket_files, best_pocket)
best_pocket_file_location

'https://proteins.plus/results/dogsite/i2AH31qHweZuwnHxJnA824UM/3w32_P_0_res.pdb'

In [15]:
# Retrieve PDB file content from URL
result = requests.get(best_pocket_file_location)
result

<Response [200]>

In [16]:
# To display the contents of the obtained PDB file  
pdb_residues = result.text
pdb_residues

'HEADER\tOutput of DoGSiteScorer by A. Volkamer\nHEADER\tPocket 0 with 288 binding site atoms written.\nHEADER\tReferences: \nHEADER\tA. Volkamer et al. Analyzing the topology of active sites: on the prediction of pockets and subpockets. J. Chem. Inf. Model. 2010,50(11), 2041-52\nHEADER\tA. Volkamer et al. Combining global and local measures for structure-based druggability predictions. J. Chem. Inf. Model. 2012,52,360-372\nHEADER\tGeometric pocket center at  13.65  29.95   8.29 with max radius 20.00\nATOM      2  CA  GLN A 701      -0.291  31.978  -3.835  0.00  0.00           C\nATOM      3  C   GLN A 701       0.946  31.062  -3.957  0.00  0.00           C\nATOM      4  O   GLN A 701       0.876  29.863  -3.659  0.00  0.00           O\nATOM      6  CG  GLN A 701      -2.441  30.619  -3.562  0.00  0.00           C\nATOM     12  C   ALA A 702       3.775  30.322  -3.161  0.00  0.00           C\nATOM     13  O   ALA A 702       3.538  30.924  -2.110  0.00  0.00           O\nATOM     15  

#### To display the best binding site residue IDs and residues

In [17]:
def residue_id(pdb_residues):
    """
    Description
    -----------
    This function displays the "residue IDs" and the "amino acid residue names" i the form of Pandas DataFrame table
    
    Parameters
    ----------
    pdb_residues : str
        
    Returns
    -------
    residue_number, residue_id : Pd dataFrame
        
    """
    ppdb = PandasPdb()
    pdb_df = ppdb._construct_df(pdb_residues.splitlines(True))['ATOM']
    return pdb_df[['residue_number', 'residue_name']]

In [18]:
#To display the Final pocket residues
final_pocket = residue_id(pdb_residues)
final_pocket

,residue_number,residue_name
0,701,GLN
1,701,GLN
2,701,GLN
3,701,GLN
4,702,ALA
5,702,ALA
6,703,LEU
7,703,LEU
8,703,LEU
9,703,LEU


# Validation using KLIFS

We have identified the binding site of the desired protein with the help of DoGSiteScorer Web Server. Now we must validate the obtained result with the help of KLIFS database 

In [19]:
#KLIFS API calling "Swagger Client"
from bravado.client import SwaggerClient
KLIFS_API_DEFINITIONS = "http://klifs.vu-compmedchem.nl/swagger/swagger.json"
KLIFS_CLIENT = SwaggerClient.from_url(KLIFS_API_DEFINITIONS, config={'validate_responses': False})

#### Obtaining the structure ID for the respective PDB ID 

In [20]:
def structures_pdb_list(pdb_code):
    """
    Description
    -----------
    This function takes the input in the form of PDB ID and returns the structure ID as the output.
    
    Parameters
    ----------
    pdb_code : str
    
    Returns
    -------
    binding_site : str
    structure_final : str
    
    """
    binding_site = KLIFS_CLIENT.Structures.get_structures_pdb_list(pdb_codes = pdb_code).response().result
    return binding_site

pdb_code = ['3w32']
structure_id_list = structures_pdb_list(pdb_code)
structure_final = str(structure_id_list[0]).split('=')[-1][:-1]
structure_final

'784'

#### To obtain the structure residues from the structure IDs

In [21]:
def protein_binding_site(structure_final):
    """
    Description
    -----------
    This function takes structure ID aqs the input and returns the binding site residues as the output
    Parameters
    ----------
    structure_final : str
    
    Returns
    -------
    binding_site = str
    binding_site_split : list
    
    """    
    protein_binding = KLIFS_CLIENT.Structures.get_structure_get_pocket(structure_ID = int(structure_final)).response().result
    return protein_binding

binding_site = protein_binding_site(structure_final)
binding_site_split = [i.split('\t') for i in binding_site[:-1].split('\n')]
binding_site_split

[['@<TRIPOS>MOLECULE'],
 ['HUMAN/EGFR_3w32_chainA'],
 ['1364 1368 85 0 0 '],
 ['BIOPOLYMER'],
 ['USER_CHARGES'],
 [''],
 [''],
 ['@<TRIPOS>ATOM'],
 ['   1 N        7.7187    15.4685    51.5111 N.3    1 LYS716   0.0000 BACKBONE'],
 ['   2 H        7.3300    16.1756    50.9036 H      1 LYS716   0.0000 BACKBONE'],
 ['   3 CA       7.3282    14.0740    51.3137 C.3    1 LYS716   0.0000 BACKBONE'],
 ['   4 HA       8.1219    13.4452    51.7172 H      1 LYS716   0.0000 BACKBONE'],
 ['   5 C        7.1509    13.7803    49.8284 C.2    1 LYS716   0.0000 BACKBONE'],
 ['   6 O        6.8337    14.6724    49.0417 O.2    1 LYS716   0.0000 BACKBONE'],
 ['   7 CB       6.0216    13.7553    52.0569 C.3    1 LYS716   0.0000 '],
 ['   8 HB2      5.6643    12.7873    51.7056 H      1 LYS716   0.0000 '],
 ['   9 HB3      5.2944    14.5258    51.8006 H      1 LYS716   0.0000 '],
 ['  10 CG       6.1220    13.6938    53.5737 C.3    1 LYS716   0.0000 '],
 ['  11 HG2      5.1238    13.8138    53.9949 H      1 

In [22]:
from biopandas.mol2 import PandasMol2

In [23]:
def residue_id(binding_site):
    """
    Description
    -----------
    This function returns the residue IDs and residue names which are returned from the KLIFS database 
    
    Parameters
    ----------
    binding_site : str
    col_names : list
    col_types : list
    
    Returns
    -------
    mol2_df[['Residues']] : list
    
        
    """
    ppdb = PandasMol2()
    mol2_df = ppdb._construct_df(
        binding_site.splitlines(True),  
        col_names=['1','2','3','4','5','6','7','Residues','9','10'],
        col_types=[str]*10
    )
    return mol2_df[['Residues']]

In [24]:
# Displays the residue IDs and residue_names retrieved from KLIFS database
final_KLIFS = residue_id(binding_site)
final_KLIFS

,Residues
0,LYS716
1,LYS716
2,LYS716
3,LYS716
4,LYS716
5,LYS716
6,LYS716
7,LYS716
8,LYS716
9,LYS716
